In [2]:
!pip install transformers
!pip install datasets

In [3]:
!pip install peft
!pip install accelerate
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:0000:0100:01


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
#, wandb, platform, gradio, warnings
from datasets import load_dataset
from huggingface_hub import notebook_login

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset

# Loading dataset

In [8]:
notebook_login()

# 

In [9]:
dataset = load_dataset("nyu-mll/multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [10]:
dataset['train']
# ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse']
#[['promptID']]

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 392702
})

In [11]:
# model_name = "meta-llama/Meta-Llama-3-8B"
model_name = "roberta-large"

dataset_name = "nyu-mll/multi_nli"

> # Reducing the test dataset to 10 percent of train dataset

In [28]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

# Load the dataset
dataset = load_dataset("nyu-mll/multi_nli")

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

# Preprocessing function to tokenize the text
def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=128)

# Map preprocess function to the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Reduce the training dataset to 10% of its size
train_size = int(0.10 * len(encoded_dataset['train']))  
train_dataset_reduced = encoded_dataset['train'].shuffle(seed=42).select(range(train_size)) 

# Function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    save_strategy="epoch",           # Model is saved at the end of each epoch
    learning_rate=2e-5,              # Learning rate (you can tweak this)
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True,     # Load the best model at the end of training
    metric_for_best_model='accuracy' # Use accuracy as the metric for evaluating the best model
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_reduced,
    eval_dataset=encoded_dataset['validation_matched'],
    compute_metrics=compute_metrics   # Compute metrics function to calculate accuracy
)

# Start training
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.112600,1.099171,0.318186
2,1.092900,1.100138,0.327356
3,1.094500,1.080447,0.394702


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=7365, training_loss=1.1018809591113348, metrics={'train_runtime': 7637.3802, 'train_samples_per_second': 15.425, 'train_steps_per_second': 0.964, 'total_flos': 2.744780121844992e+16, 'train_loss': 1.1018809591113348, 'epoch': 3.0})

# RobertaLarge Model with Lora Config

In [30]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model  



dataset = load_dataset("nyu-mll/multi_nli")


tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)


peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",  
)

# Apply LoRA modifications to the model
model = get_peft_model(model, peft_config)

# Preprocessing function to tokenize the text
def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=128)

# Map preprocess function to the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Reduce the training dataset to 10% of its size
train_size = int(0.10 * len(encoded_dataset['train']))
train_dataset_reduced = encoded_dataset['train'].shuffle(seed=42).select(range(train_size))

# Function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}


# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_reduced,
    eval_dataset=encoded_dataset['validation_matched'],
    compute_metrics=compute_metrics
)

# Start training
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.085200,1.070879,0.394396
2,0.955700,0.919808,0.606521
3,0.857600,0.803203,0.681406


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=7365, training_loss=0.995622140990579, metrics={'train_runtime': 5859.2636, 'train_samples_per_second': 20.107, 'train_steps_per_second': 1.257, 'total_flos': 2.759011093327104e+16, 'train_loss': 0.995622140990579, 'epoch': 3.0})

# RobertaLarge Model with P_tuning Config

In [15]:
!pip install safetensors

In [28]:
# import time

# # Load model
# model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=3)

# # Configure P-Tuning
# p_tuning_config = PromptEncoderConfig(
#     task_type='SEQ_CLS',
#     num_virtual_tokens=10,  
#     encoder_hidden_size=model.config.hidden_size,
# )

# # Integrate P-Tuning with the model
# model = get_peft_model(model, p_tuning_config)
# model.to(device)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results-p-tuning',
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     logging_dir='./logs-p-tuning',
#     logging_steps=500,
#     fp16=True
# )

# # Initialize trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_dataset['train'],
#     eval_dataset=encoded_dataset['validation_matched'],
#     tokenizer=tokenizer
# )

# # Measure training time
# start_time = time.time()
# trainer.train()
# training_time = time.time() - start_time

# # Evaluate model
# eval_results = trainer.evaluate()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all 

Epoch,Training Loss,Validation Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.roberta.embeddings.word_embeddings.weight', 'word_embeddings.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [25]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
from peft import get_peft_model, PromptEncoderConfig

dataset = load_dataset("nyu-mll/multi_nli")

tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

peft_config = PromptEncoderConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    encoder_hidden_size=model.config.hidden_size
)

model = get_peft_model(model, peft_config)

def preprocess_function(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation = "only_first",
        padding="max_length",
        max_length=128 ,
        return_overflowing_tokens=True,
        stride = 0
    )

encoded_dataset = dataset.map(preprocess_function, batched=True)

train_size = int(0.10 * len(encoded_dataset['train']))
train_dataset_reduced = encoded_dataset['train'].shuffle(seed=42).select(range(train_size))

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_reduced,
    eval_dataset=encoded_dataset['validation_matched'],
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all 

Epoch,Training Loss,Validation Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.roberta.embeddings.word_embeddings.weight', 'word_embeddings.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            